# GMS Intro to Stats
## Excercise 1: Tests of significance and correlations
### Looking at the minor allele frequency for 5000 introgressed neaderthal SNPs in modern humans

#### Neanderthal SNPs from:
1.	Dannemann M, Prufer K & Kelso J. Functional implications of Neandertal introgression in modern humans. *Genome Biol* 2017 18:61.
2.	Simonti CN et al. The phenotypic legacy of admixture between modern humans and Neandertals. *Science* 2016 351:737-41.

#### Minor allele frequencies from:
+ [1000 genomes](http://www.internationalgenome.org/data)
    - [With five super populatuions and 26 specific populations](http://www.internationalgenome.org/category/population/)


In [ ]:
# Import the packages we need
# Manipulate the data
import pandas as pd
import numpy as np
import scipy as sp
# Plot the data
import matplotlib.pyplot as plt
from pylab import plot, show, savefig, xlim, figure, ylim, legend, boxplot, setp, axes
import seaborn as sns
sns.set()

In [ ]:
# Import the data - European 
eur = pd.read_csv('neanderthal_snps_european_maf_1000genome.tsv', index_col=0, sep='\t')

In [ ]:
# Summary statistics
eur.describe()

In [ ]:
# What does it look like?
sns.histplot(eur, kde=True)

In [ ]:
# Looks normal, is it normal?
print(sp.stats.normaltest(eur))

In [ ]:
# Now lets import the African MAFs
afr = pd.read_csv('neanderthal_snps_african_maf_10000genome.tsv', index_col=0, sep='\t')

In [ ]:
sns.distplot(afr, color='red')

In [ ]:
print(sp.stats.normaltest(afr))

+ We want to compare the European and African MAFs for the neanderthal SNPs:
    - What is our hypothesis?
    - What would lead us to reject this?

In [ ]:
# How about non-parametric tests
# print('Mann-Whitney rank test')
# print(sp.stats.mannwhitneyu(eur,afr))
# print('Kruskal-Wallis test')
# print(sp.stats.ks_2samp(eur['European'].values,afr['African'].values))
print('Wilcoxon signed rank test')
print(sp.stats.wilcoxon(eur['European'],afr['African']))

 + The European minor allele frequencies is significantly 'bigger' than the African minor allele frequencies.
     - Is this what we expected? What do we mean by significant?
 + Are they correlated?

In [ ]:
# Work out by hand the correlation between European and African MAF for neanderthal SNPs
eur_afr = pd.concat([eur, afr], axis=1)
mean_eur = np.mean(eur_afr['European'])
mean_afr = np.mean(eur_afr['African'])

eur_afr['Eur - mean_eur'] = eur_afr['European'] - mean_eur
eur_afr['Afr - mean_afr'] = eur_afr['African'] - mean_afr
eur_afr['E-m_e * A-a_e'] = eur_afr['Eur - mean_eur'] * eur_afr['Afr - mean_afr']
eur_afr['Eur - mean_eur ^2'] = eur_afr['Eur - mean_eur']**2
eur_afr['Afr - mean_afr ^2'] = eur_afr['Afr - mean_afr']**2

corr_coeff = sp.sum(eur_afr['E-m_e * A-a_e']) / (np.sqrt(sum(eur_afr['Eur - mean_eur ^2'])) * np.sqrt(sum(eur_afr['Afr - mean_afr ^2'])))
print(corr_coeff)

In [ ]:
# Or just work out, including Spearmans rank coefficient
print(sp.stats.pearsonr(eur['European'].values,afr['African'].values))
print(sp.stats.spearmanr(eur,afr))

In [ ]:
all_pop = pd.read_csv('neanderthal_snps_maf_10000genome.tsv', index_col=0, sep='\t')
all_pop[['African', 'American', 'East_Asian', 'European', 'South_Asian']].describe()

In [ ]:
# Define the correlation coefficient so we can plot it for each comparison
def corrfunc(x, y, **kws):
    r, _ = sp.stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate(u"\u03C1 = {:.2f}".format(r), #unicode code for lowercase rho (ρ)
                xy=(.1, .9), xycoords=ax.transAxes)

# Plot a pairplot comparing the MAFs for the super populations
g = sns.pairplot(all_pop[['African', 'American', 'East_Asian', 'European', 'South_Asian']], diag_kind="kde")
g.map_lower(corrfunc)
plt.show()

In [ ]:
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(eur['European'].values,afr['African'].values)
print("slope: %f    intercept: %f" % (slope, intercept))

+ Why are any neanderthal SNPs seen in African populations (MAF > 0)?
    - Is this true for the African subpopulations in 1000 genomes?

In [ ]:
# Import the individual African population MAF for the neanderthal SNPs?
afr_pop = pd.read_csv('neanderthal_snps_african_specific_maf_10000genome.tsv', index_col=0, sep='\t')
afr_pop[['Yoruba', 'Luhya', 'Gambian', 'Mende', 'Esan', 'American', 'Caribbean']].describe()

In [ ]:
# The range of MAFs for the African populations is nicely shown by a boxplot
xticklabs = ['Yoruba', 'Luhya', 'Gambian', 'Mende', 'Esan', 'American', 'Caribbean']
colors=['red','green','blue','gold', 'orange', 'brown', 'violet', 'gray']
fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111)
for i,item in enumerate(xticklabs):
    # All
    values = afr_pop[item].values
    
    color = colors[i]
    def setBoxColors(bp):
        setp(bp['boxes'][0], color=color)
        setp(bp['caps'][0], color=color)
        setp(bp['caps'][1], color=color)
        setp(bp['whiskers'][0], color=color)
        setp(bp['whiskers'][1], color=color)
        setp(bp['medians'][0], color=color)
    bp = ax.boxplot(values, positions = [(i)], widths=0.4)
    setBoxColors(bp)
    x = np.random.normal((i), 0.04, size=len(values))
#     pn = ax.scatter(x, values, marker='.', color=color, alpha=0.2)


ax.set_xticklabels(xticklabs) 
ax.set_xticks(range(len(xticklabs)))
ax.set_xlim(-1,len(xticklabs))

ax.set_ylabel('Allele freq')

In [ ]:
# Also look at their correlations
g = sns.pairplot(afr_pop[['Yoruba', 'Luhya', 'Gambian', 'Mende', 'Esan', 'American', 'Caribbean']])
g.map_lower(corrfunc)
plt.show()